In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib notebook

from configobj import ConfigObj
from configobj import flatten_errors
from validate import Validator
    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

np.set_printoptions(precision=2)

In [2]:
fileTimeList = [];
[fileTimeList.append(name) for name in os.listdir("../Data/") if name[0] is not '.'];
fileTimeList

['2018-4-30-13-49',
 '2018-4-26-11-31',
 '2018-4-27-12-32',
 '2018-4-30-15-4',
 '2018-4-26-15-49',
 '2018-4-27-11-16',
 '2018-4-26-12-30',
 '2018-4-27-13-41']

In [3]:
from analysisParameters import loadParameters
from loadDataE1 import loadSessionDict
from assessTrackQualityE1 import calcCalibrationQuality, plotTrackQuality

sessionFiles = []
for subNum, fileName in enumerate(fileTimeList):
    
    analysisParameters = loadParameters(fileName)

    # set loadProcessedData to True if you want to reprocess the file.  
    sessionDict = loadSessionDict(analysisParameters,
                                  loadParsedData = True,
                                  loadProcessedData = True)
    
    sessionDict.pop('calibrationData', None)
    
    sessionDict = calcCalibrationQuality(sessionDict)
    sessionDict['calibrationData']['fileName'] = sessionDict['analysisParameters']['fileName']
    #plotTrackQuality(sessionDict, saveFig = True)

    sessionFiles.append(sessionDict)


INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-30-13-49 ***
INFO_assessTrackQualityE1-calcCycEIH(): - Added sessionDict['processed']['cycEyeInHead']
INFO_assessTrackQualityE1-calcCalibrationVectors(): - Added sessionDict['calibration']['targetInHead_az']
INFO_assessTrackQualityE1-calcCalibrationVectors(): - Added sessionDict['calibration']['targetInHead_el']
INFO_assessTrackQualityE1-calcCalibrationVectors(): - Added sessionDict['calibration']['cycEyeInHead_az']
INFO_assessTrackQualityE1-calcCalibrationVectors(): - Added sessionDict['calibration']['cycEyeInHead_el']
INFO_loadDataE1-loadSessionDict(): - Loading preprocessed data for exp_data-2018-4-26-11-31 ***
INFO_assessTrackQualityE1-calcCycEIH(): - Added sessionDict['processed']['cycEyeInHead']
INFO_assessTrackQualityE1-calcCalibrationVectors(): - Added sessionDict['calibration']['targetInHead_az']
INFO_assessTrackQualityE1-calcCalibrationVectors(): - Added sessionDict['calibration']['targetInHead

In [4]:
sessionDict['analysisParameters']['fileName']

'exp_data-2018-4-27-13-41'

In [5]:
sessionDict.keys()

['calibrationData',
 'expConfig',
 'calibration',
 'practice',
 'raw',
 'analysisParameters',
 'processed',
 'sysCfg',
 'trialInfo']

In [6]:
allCalibDataDf = sessionFiles[0]['calibrationData']

for subNum in range(1,len(sessionFiles)):
    allCalibDataDf = pd.concat([allCalibDataDf, sessionFiles[subNum]['calibrationData']])
allCalibDataDf = allCalibDataDf.reset_index()

In [7]:
allCalibDataDf['targetPos_az'] = np.around(allCalibDataDf['targetPos_az'],2)
allCalibDataDf['targetPos_el'] = np.around(allCalibDataDf['targetPos_el'],2)
azEl_target = zip(allCalibDataDf['targetPos_az'],allCalibDataDf['targetPos_el'])
targetNum = np.zeros(len(allCalibDataDf))
for tNum, az_el in enumerate(np.unique(set(azEl_target))[0]):
    #idx = np.where(allCalibDataDf[ (allCalibDataDf['targetPos_az']==az_el[0]) & (allCalibDataDf['targetPos_el']==az_el[1]) ])[0]
    idx = allCalibDataDf[ (allCalibDataDf['targetPos_az']==az_el[0]) & (allCalibDataDf['targetPos_el']==az_el[1]) ].index
    targetNum[idx] = tNum

allCalibDataDf['targetNum'] = targetNum

In [8]:
from scipy.stats import sem
allCalibDataDf.groupby('targetNum').agg([np.mean,sem])

#calibDf_sub['meanCalibError'].agg([np.mean,lambda x: np.std(x)/np.sqrt(len(x)) ])

index     gazePos_az           gazePos_el           meanCalibError  \
           mean sem       mean       sem       mean       sem           mean   
targetNum                                                                      
0.0           4   0  -0.699724  0.132725  -0.670235  0.356657       0.879874   
1.0           8   0  11.994815  0.218063  11.977110  0.163135       0.681695   
2.0           6   0 -12.613526  0.316149  11.866199  0.196075       1.185609   
3.0           1   0  -0.666051  0.120658 -12.745860  0.624375       1.422814   
4.0           5   0  11.653674  0.275364  -0.930628  0.319210       1.025071   
5.0           7   0  -0.430188  0.103503  11.469012  0.204663       0.692325   
6.0           3   0 -12.333235  0.334434  -0.997675  0.349948       1.260325   
7.0           0   0 -12.612291  0.136496 -13.048939  0.592517       1.392582   
8.0           2   0  11.386368  0.235275 -13.375117  0.753205       1.615258   

                    targetPos_az      targetPos_el       
                sem         mean  sem         mean  sem  
targetNum                                                
0.0        0.227880         -0.5  0.0        -0.50  0.0  
1.0        0.210514         12.0  0.0        12.26  0.0  
2.0        0.283065        -13.0  0.0        12.31  0.0  
3.0        0.421493         -0.5  0.0       -13.00  0.0  
4.0        0.270807         12.0  0.0        -0.51  0.0  
5.0        0.172928         -0.5  0.0        12.00  0.0  
6.0        0.362187        -13.0  0.0        -0.51  0.0  
7.0        0.337987        -13.0  0.0       -13.33  0.0  
8.0        0.531510         12.0  0.0       -13.28  0.0

In [9]:
allCalibDataDf.groupby('fileName').agg([np.mean,np.std])

index           gazePos_az            gazePos_el  \
                          mean       std       mean        std       mean   
fileName                                                                    
exp_data-2018-4-26-12-30     4  2.738613  -0.432586  10.270973  -0.630802   
exp_data-2018-4-26-15-49     4  2.738613  -0.196271  10.360016  -0.365984   
exp_data-2018-4-27-11-16     4  2.738613  -0.385360  10.626596  -0.451549   
exp_data-2018-4-27-12-32     4  2.738613  -0.328854  11.536033  -2.953765   
exp_data-2018-4-27-13-41     4  2.738613  -0.580182   9.801790  -0.521000   
exp_data-2018-4-30-13-49     4  2.656845  -0.610916  10.118508  -0.156476   
exp_data-2018-4-30-15-4      4  2.738613  -0.695055  10.446142  -0.502732   

                                    meanCalibError           targetPos_az  \
                                std           mean       std         mean   
fileName                                                                    
exp_data-2018-4-26-12-30  10.685017       0.881214  0.735342         -0.5   
exp_data-2018-4-26-15-49  10.258117       0.949951  0.344833         -0.5   
exp_data-2018-4-27-11-16  10.717711       0.437820  0.162455         -0.5   
exp_data-2018-4-27-12-32  12.605280       2.839045  1.208447         -0.5   
exp_data-2018-4-27-13-41  10.304609       1.418648  0.899293         -0.5   
exp_data-2018-4-30-13-49  10.128679       0.899170  0.419886         -0.5   
exp_data-2018-4-30-15-4   10.651837       0.702140  0.303794         -0.5   

                                    targetPos_el            targetNum  \
                                std         mean        std      mean   
fileName                                                                
exp_data-2018-4-26-12-30  10.825318    -0.506667  10.996311       4.0   
exp_data-2018-4-26-15-49  10.825318    -0.506667  10.996311       4.0   
exp_data-2018-4-27-11-16  10.825318    -0.506667  10.996311       4.0   
exp_data-2018-4-27-12-32  10.825318    -0.506667  10.996311       4.0   
exp_data-2018-4-27-13-41  10.825318    -0.506667  10.996311       4.0   
exp_data-2018-4-30-13-49  10.502101    -0.506667  10.667988       4.0   
exp_data-2018-4-30-15-4   10.825318    -0.506667  10.996311       4.0   

                                    
                               std  
fileName                            
exp_data-2018-4-26-12-30  2.738613  
exp_data-2018-4-26-15-49  2.738613  
exp_data-2018-4-27-11-16  2.738613  
exp_data-2018-4-27-12-32  2.738613  
exp_data-2018-4-27-13-41  2.738613  
exp_data-2018-4-30-13-49  2.656845  
exp_data-2018-4-30-15-4   2.738613